# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Weno,7.4515,151.8468,82.35,83,100,4.61,1007,FM,1732758669
1,1,George Town,5.4112,100.3354,76.95,87,40,9.22,1011,MY,1732758395
2,2,Brisas de Zicatela,15.8369,-97.0419,81.12,83,0,0.00,1011,MX,1732758672
3,3,Klyuchi,52.2667,79.1667,15.96,81,3,7.61,1029,RU,1732758674
4,4,Pugachev,52.0133,48.8025,28.54,68,99,5.48,1024,RU,1732758675


In [4]:
# Date Cleaning w/dates
city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Weno,7.4515,151.8468,82.35,83,100,4.61,1007,FM,2024-11-28 01:51:09
1,1,George Town,5.4112,100.3354,76.95,87,40,9.22,1011,MY,2024-11-28 01:46:35
2,2,Brisas de Zicatela,15.8369,-97.0419,81.12,83,0,0.00,1011,MX,2024-11-28 01:51:12
3,3,Klyuchi,52.2667,79.1667,15.96,81,3,7.61,1029,RU,2024-11-28 01:51:14
4,4,Pugachev,52.0133,48.8025,28.54,68,99,5.48,1024,RU,2024-11-28 01:51:15


In [5]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     564 non-null    int64         
 1   City        564 non-null    object        
 2   Lat         564 non-null    float64       
 3   Lng         564 non-null    float64       
 4   Max Temp    564 non-null    float64       
 5   Humidity    564 non-null    int64         
 6   Cloudiness  564 non-null    int64         
 7   Wind Speed  564 non-null    float64       
 8   Pressure    564 non-null    int64         
 9   Country     560 non-null    object        
 10  Date        564 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 48.6+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
city_data_df2 = city_data_df.dropna()

mask = (city_data_df2["Max Temp"] >= min_temp) & (city_data_df2["Max Temp"] < max_temp) & (city_data_df2["Wind Speed"] < max_wind)
city_data_df2 = city_data_df2.loc[mask].reset_index(drop=True)

# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,1,George Town,5.4112,100.3354,76.95,87,40,9.22,1011,MY,2024-11-28 01:46:35
1,12,Mossamedes,-15.1961,12.1522,71.06,79,86,6.13,1012,AO,2024-11-28 01:51:24
2,22,San Antonio de Palé,-1.4014,5.6325,78.58,81,91,4.88,1012,GQ,2024-11-28 01:51:36
3,23,Broken Hill,-31.9500,141.4333,76.01,37,0,2.86,1014,AU,2024-11-28 01:51:37
4,39,Puerto Ayora,-0.7393,-90.3518,71.55,94,91,5.99,1013,EC,2024-11-28 01:51:57
...,...,...,...,...,...,...,...,...,...,...,...
68,541,Itaí,-23.4178,-49.0906,75.47,63,73,2.98,1010,BR,2024-11-28 02:00:40
69,542,Lamu,-2.2717,40.9020,79.09,85,52,5.93,1011,KE,2024-11-28 02:02:31
70,553,Callao,-12.0667,-77.1500,71.73,82,40,8.05,1014,PE,2024-11-28 02:00:00
71,560,Dambulla,7.8600,80.6517,72.81,96,100,8.95,1008,LK,2024-11-28 02:02:54


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
hotel_rows = []

for index, row in city_data_df2.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [11]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,George Town,MY,5.4112,100.3354,"Page 63 hostel, Beach Street, 10300 George Tow...",Page 63 hostel,139.0,None,None
1,Mossamedes,AO,-15.1961,12.1522,"Nelsal Pensao, Rua Kahumba, Moçâmedes, Angola",Nelsal Pensao,405.0,None,None
2,San Antonio de Palé,GQ,-1.4014,5.6325,"Sendero Aual - Palé, Aual, Equatorial Guinea",None,2011.0,None,None
3,Broken Hill,AU,-31.9500,141.4333,"Ibis Styles, Argent Street, Broken Hill NSW 28...",Ibis Styles,2893.0,None,None
4,Puerto Ayora,EC,-0.7393,-90.3518,"Hostal Mirada Del Solitario George Hostel, Pet...",Hostal La Mirada De Solitario George,3776.0,None,None
...,...,...,...,...,...,...,...,...,...
68,Itaí,BR,-23.4178,-49.0906,"Hotel Nixon, Rua Expedicionários, Centro, Itaí...",Hotel Nixon,143.0,None,None
69,Lamu,KE,-2.2717,40.9020,"Hotel Archipelago villa bed and breakfast, Cor...",Hotel Archipelago villa bed and breakfast,157.0,None,None
70,Callao,PE,-12.0667,-77.1500,"Casa Ronald, Calle La Constitución, Lima Metro...",Casa Ronald,748.0,None,None
71,Dambulla,LK,7.8600,80.6517,"Dumbulla Rest House, Jaffna - Kandy Road, Mora...",Dumbulla Rest House,91.0,None,None


In [12]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name,address)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {"limit":limit,
   "categories": f"accommodation.hotel",
   "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "latitude"]
    longitude = hotel_df.loc[index, "longitude"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'city']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
George Town - nearest hotel: Page 63 hostel
Mossamedes - nearest hotel: Nelsal Pensao
San Antonio de Palé - nearest hotel: No hotel found
Broken Hill - nearest hotel: Ibis Styles
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Muisne - nearest hotel: Hostal Té de Menta
Whangarei - nearest hotel: Grand Hotel
Margaret River - nearest hotel: Margaret River Hotel
Cidreira - nearest hotel: Hotel Castelo
Pantai Remis - nearest hotel: Hotel Lam Seng
Sapucaia - nearest hotel: No hotel found
Frontera - nearest hotel: Quirino
Mirpur Sakro - nearest hotel: No hotel found
Hwange - nearest hotel: No hotel found
Afaahiti - nearest hotel: Omati Lodge
Kerikeri - nearest hotel: Avalon Resort
Road Town - nearest hotel: No hotel found
Yanam - nearest hotel: Hôtel Sarvainn
Hadibu - nearest hotel: No hotel found
Aripuanã - nearest hotel: No hotel found
Taveta - nearest hotel: Garden hotel
Pangoa - nearest hotel: No hotel found
Ceeldheer - nearest hotel: No hotel fou

,city,country,latitude,longitude,address,name,distance,elevation,website,Hotel Name
0,George Town,MY,5.4112,100.3354,"Page 63 hostel, Beach Street, 10300 George Tow...",Page 63 hostel,139.0,None,None,Page 63 hostel
1,Mossamedes,AO,-15.1961,12.1522,"Nelsal Pensao, Rua Kahumba, Moçâmedes, Angola",Nelsal Pensao,405.0,None,None,Nelsal Pensao
2,San Antonio de Palé,GQ,-1.4014,5.6325,"Sendero Aual - Palé, Aual, Equatorial Guinea",None,2011.0,None,None,No hotel found
3,Broken Hill,AU,-31.9500,141.4333,"Ibis Styles, Argent Street, Broken Hill NSW 28...",Ibis Styles,2893.0,None,None,Ibis Styles
4,Puerto Ayora,EC,-0.7393,-90.3518,"Hostal Mirada Del Solitario George Hostel, Pet...",Hostal La Mirada De Solitario George,3776.0,None,None,Hostal La Mirada De Solitario George
...,...,...,...,...,...,...,...,...,...,...
68,Itaí,BR,-23.4178,-49.0906,"Hotel Nixon, Rua Expedicionários, Centro, Itaí...",Hotel Nixon,143.0,None,None,Hotel Nixon
69,Lamu,KE,-2.2717,40.9020,"Hotel Archipelago villa bed and breakfast, Cor...",Hotel Archipelago villa bed and breakfast,157.0,None,None,Hotel Archipelago villa bed and breakfast
70,Callao,PE,-12.0667,-77.1500,"Casa Ronald, Calle La Constitución, Lima Metro...",Casa Ronald,748.0,None,None,Casa Ronald
71,Dambulla,LK,7.8600,80.6517,"Dumbulla Rest House, Jaffna - Kandy Road, Mora...",Dumbulla Rest House,91.0,None,None,Dumbulla Rest House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "humidity",
    scale = 1,
    color = "city",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,Hotel Name)